In [1]:
import csv
import pandas as pd
import os

In [2]:
FILE_PATH = r'D:\HOANG ANH\HK232\Capstone\capstone_project\backend\storage\ecom_data.csv'

def readCsvFile(filepath: str) -> pd.DataFrame:
    # Read the CSV file
    filepath = os.path.normpath(filepath) 
    df = pd.read_csv(filepath)
    return df

df_full = readCsvFile(FILE_PATH)
df_full.head()

,order_number,order_line_number,order_date,order_time,customer_key,channel_name,store_key,product_key,unit_price,order_quantity,...,is_active,product_name,product_category,product_subcategory,uom,price,cost,country,brand,suplier_name
0,SO20230228/TIKI-15,1,2023-02-28,14:13:01.0000000,9163,Tiki,2,92,363000,2,...,1,Miến khác,Thực phẩm đóng gói,Thực phẩm ăn liền,Thùng,363000,225641,Việt Nam,Vifon,Vifon
1,SO20230228/TIKI-16,1,2023-02-28,15:03:36.0000000,8479,Tiki,18,6,13300,14,...,1,Ngũ vị hương,Gia vị - nguyên liệu nấu ăn,Gia vị dạng bột,Hũ,13300,4974,Việt Nam,Thiên Thành,Thuận Phát
2,SO20230228/TIKI-16,2,2023-02-28,15:03:36.0000000,8479,Tiki,18,81,580000,3,...,1,Nồi nhôm,Đồ gia dụng,Đồ dùng nhà bếp,Bộ,580000,369112,Việt Nam,Sunhouse,Sunhouse
3,SO20230228/TIKI-16,3,2023-02-28,15:03:36.0000000,8479,Tiki,18,2,12500,4,...,1,Khăn giấy bỏ túi,Đồ gia dụng,Đồ sử dụng một lần,Gói,12500,6250,Hàn Quốc,Paseo,Paseo
4,SO20230228/TIKI-16,4,2023-02-28,15:03:36.0000000,8479,Tiki,18,97,270000,4,...,1,Cafe sữa lon,Thức uống,Café,Thùng,270000,135000,Việt Nam,Georgia,Gama


# Connect to DB KPIM Test

In [9]:
import pyodbc

In [9]:

# Database connection information
server = '222.252.14.117'
database = 'kpim_mart_v2'
username = 'bi7'
password = 'BI7@2023'
driver = '{SQL Server}' # Adjust the driver based on your SQL Server version

# Create the connection string
connection_string = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'

# Connect to the database
try:
    conn = pyodbc.connect(connection_string)
    print("Connection successful")
except Exception as e:
    print(f"Error connecting to the database: {e}")
    exit()

# Perform a query
try:
    cursor = conn.cursor()
    # cmd = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';"
    # cmd = "SELECT * FROM dim_product;"
    # cmd = "SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'fact_ecommerce_sales';"
    # cmd = "SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'dim_product';"
    cmd = generateAggregationQueryCard('order_quantity', 'SUM')
    cursor.execute(cmd) # Replace 'YourTableName' with the actual table name
    rows = cursor.fetchall()
    for row in rows:
        print(row)
except Exception as e:
    print(f"Error executing query: {e}")

# Close the connection
conn.close()


Error connecting to the database: name 'pyodbc' is not defined
(Decimal('17039320'),)


In [14]:
numeric_fields = [
                'date_key', 'time_key', 'channel_key', 'customer_key', 'product_key',
                'store_key', 'promotion_key', 'order_quantity', 'unit_price', 'unit_cost',
                'unit_discount', 'total_sale', 'order_line_number'
                ]

aggregation_options = ['SUM', 'AVERAGE', 'COUNT', 'DISTINCT']

In [13]:
def generateAggregationQueryCard(field: str, aggre: str):
    if field not in numeric_fields and aggre in ['SUM', 'AVERAGE']:
        return 'This type of aggregation is not supported for this field'
    
    if aggre == 'SUM':
        query = f'SELECT SUM({field}) FROM fact_ecommerce_sales;'
    elif aggre == 'AVERAGE':
        query = f'SELECT AVG({field}) FROM fact_ecommerce_sales;'
    elif aggre == 'COUNT':
        query = f'SELECT COUNT({field}) FROM fact_ecommerce_sales;'
    elif aggre == 'DISTINCT':
        query = f'SELECT COUNT(DISTINCT {field}) FROM fact_ecommerce_sales;'
    else: 
        query = 'Invalid aggregation option'
    
    return query


In [15]:
import pymysql

# Database connection information
server = 'localhost'
database = 'dw'
username = 'root'
password = 'root'

# Create the connection string
connection_string = {
    'host': server,
    'user': username,
    'password': password,
    'database': database
}

# Connect to the database
try:
    conn = pymysql.connect(**connection_string)
    print("Connection successful")
except Exception as e:
    print(f"Error connecting to the database: {e}")
    exit()

# Perform a query
try:
    cursor = conn.cursor()
    # cmd = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';"
    # cmd = "SELECT * FROM dim_product;"
    # cmd = "SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'fact_ecommerce_sales';"
    # cmd = "SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'dim_product';"
    cmd = generateAggregationQueryCard('total_sale', 'SUM')
    cursor.execute(cmd) # Replace 'YourTableName' with the actual table name
    rows = cursor.fetchall()
    print(rows)
    print('---------------------------------')
    print(rows[0][0])
except Exception as e:
    print(f"Error executing query: {e}")


Connection successful
((Decimal('5628676132100'),),)
---------------------------------
5628676132100
